import pandas as pd
from scipy.stats import pearsonr

# Assuming your data is in a DataFrame named 'df' with columns 'COST_log' and 'FREE TRIALS_log'

# Calculate the Pearson correlation coefficient
corr_coef, p_value = pearsonr(df['COST'], df['FREE TRIALS_log'])

print(f"Pearson correlation coefficient: {corr_coef}")
print(f"P-value: {p_value}")

# Interpret the results
if abs(corr_coef) > 0.7:
    print("Strong correlation")
elif abs(corr_coef) > 0.3:
    print("Moderate correlation")
else:
    print("Weak correlation")

if p_value < 0.05:
    print("The correlation is statistically significant.")
else:
    print("The correlation is not statistically significant.")